In [0]:
%python
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")
 

In [0]:
%python
df_customer = spark.table("samples.tpch.customer")
df_orders = spark.table("samples.tpch.orders")

In [0]:
%python
customer_path = "/Volumes/workspace/default/customer_orders_csv/customer.csv"
orders_path = "/Volumes/workspace/default/customer_orders_csv/orders.csv"

df_customer_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(customer_path)
)

df_orders_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(orders_path)
)

display(df_customer_csv)
display(df_orders_csv)

In [0]:
%python
df_customer_csv.write.mode("overwrite").saveAsTable("workspace.default.customer_csv")
df_orders_csv.write.mode("overwrite").saveAsTable("workspace.default.orders_csv")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.default.etl_run_log (
  run_id STRING,
  status STRING,
  message STRING,
  log_date DATE
)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.default.customer_transformation (
  c_custkey STRING,
  c_name STRING,
  c_address STRING,
  c_nationkey STRING,
  c_phone STRING,
  c_acctbal STRING,
  c_mktsegment STRING,
  c_comment STRING,
  load_date DATE,
  run_id STRING);
  
  CREATE TABLE IF NOT EXISTS workspace.default.orders_transformation (
  o_orderkey STRING,
  o_custkey STRING,
  o_orderstatus STRING,
  o_totalprice STRING,
  o_orderdate STRING,
  o_orderpriority STRING,
  o_clerk STRING,
  o_shippriority STRING,
  o_comment STRING,
  load_date DATE,
  run_id STRING) ;

In [0]:
%python
 
orders_path = "/Volumes/workspace/default/customer_orders_csv/orders.csv"


df_orders_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(orders_path)
)

df_orders_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .load(orders_path)
)

df_orders_csv.printSchema()
df_orders_csv.show(10) 

In [0]:
selected_order_columns = df_orders_csv.select(
    "o_orderkey",
    "o_custkey",
    "o_orderstatus",
    "o_totalprice",
    "o_orderdate",
    "o_orderpriority"
)

selected_order_columns.show(10)

In [0]:
%python
filter_selected_order_columns = df_orders_csv.filter(
    (df_orders_csv.o_orderdate >= "1995-01-01") & (df_orders_csv.o_orderstatus == "F")
).select(
    "o_orderkey",
    "o_custkey",
    "o_orderstatus",
    "o_totalprice",
    "o_orderdate",
    "o_orderpriority"
)

filter_selected_order_columns.show(10)

In [0]:
%python
group_data = df_orders_csv.groupBy("o_orderpriority").count()
display(group_data)

In [0]:
%python
group_data_sum = df_orders_csv.groupBy("o_orderpriority").agg({"o_totalprice": "sum"})
group_data_avg = df_orders_csv.groupBy("o_orderpriority").agg({"o_totalprice": "avg"})
display(group_data_sum)
display(group_data_avg)

group_data_sum_avg = group_data_sum.join(group_data_avg, "o_orderpriority")
display(group_data_sum_avg)

In [0]:
from pyspark.sql.functions import col

sotered_df = filter_selected_order_columns.orderBy(
    "o_orderpriority",
    col("o_orderdate").desc()
)
display(sotered_df)

In [0]:
%python
distinct_df = filter_selected_order_columns.select(
    "o_orderpriority","o_orderstatus"
).distinct()
display(distinct_df)

In [0]:
drop_columns = filter_selected_order_columns.drop(
    "o_orderpriority","o_orderstatus"
)

display(drop_columns)

In [0]:
add_columns = filter_selected_order_columns.withColumn(
    "more10perct", col("o_totalprice") * 1.01
    
)

display(add_columns)

In [0]:
columns_remnamed = filter_selected_order_columns.withColumnRenamed("o_totalprice", "total_price")

display(columns_remnamed)